In [1]:
import dao.MongoTagUser as mtu
import panda.TagUser as tupd

mongotu = mtu.TagofSongUserRecom(user_id="1a78ac10b67911ed95dd00155dadb10b")
# list = mongotu.makeRecomUsersSet()
# print(list)

In [2]:
doc = mongotu.rdb.findDocument(
            collection_name="like", query=mongotu.query, projection=mongotu.projections)
# 储存请求用户的名字
mongotu.user_name = doc[0]['name']
# 随机获取一定数量的用户来计算tag 相似度
# 统一获取其他用户的信息
querys = [{'$sample': {"size": 50-1}}]
project = {'$project': mongotu.a_projections}
querys.append(project)
docs = mongotu.rdb.aggregateDocument(
     collection_name="like", querys=querys)
print(doc)

[{'id': '1a78ac10b67911ed95dd00155dadb10b', 'name': 'RuRuUIH', 'tags': ['爵士', '驾车', '古典', '90后', '民族', '伤感', '安静', '钢琴', '雷鬼', '学习', '怀旧', '夜晚', '拉丁', '轻音乐', '英伦', '治愈', 'R&B/Soul', '快乐', '孤独', '放松', 'Bossa Nova', '古风', '说唱', '游戏', '金属', '小语种', '吉他', '粤语', '兴奋', '舞曲', '思念', '影视原声', '朋克', '清新', '经典', '后摇', 'New Age', '日语', '流行', '乡村', '性感', '酒吧', '蓝调', '世界音乐', '民谣', '韩语', '欧美', '摇滚', '另类/独立', '浪漫', '感动', '电子', 'ACG', '旅行', '下午茶', '华语', '器乐', '运动']}]


In [3]:
# 判断 docs中是否存在 doc 并合并
docs = mongotu.isDocinDocs(doc=doc, docs=docs)
# print(docs)     

In [4]:
# 查找数据songs 便于 统计tags 出现的次数
# 提取docs 中用户的id，之后用
list_name = mongotu.getItemFromDocs(docs=docs, key='name')
list_id = mongotu.getItemFromDocs(docs=docs, key='id')
# 通过这个方法获取songs，来计算tag出现次数
docs_song = mongotu.getSongsforTagfromMongo(list_id)
# count_tag 统计
dict_tags = mongotu.makeMatricTagCountData(docs=docs_song)
# print(dict_tags)
        

In [5]:
# 根据以上docs 来生成 xy轴，矩阵
# 内容
# matrix_data = data
# pandas_x = self.x
# pandas_y = self.y
data = mongotu.makeXYMatric(docs=docs)
# print(data)
        

In [6]:
# 生成pandas 表格
tudf = tupd.TUPandas(datas=list(
map(list, data.toarray())), items=mongotu.y, users=mongotu.x)



In [7]:
tudf.df 

,RuRuUIH,EwNjz,d,RgxKB,ZcO,G,zhZd1J1,79vexmSD,H6fLkSvrlg,c4D,...,PdHcku8A,axRJKQiq5,kRM4zo,DSWI5CjX,FHOCydFdKN,HrlIEqGS,IDDRfNJpw,m5eKRJ,2,gW
小语种,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
说唱,1,1,1,1,1,1,1,1,1,1,...,1,1,1,0,1,1,1,1,1,1
民族,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
治愈,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
感动,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
兴奋,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,0,1,1
民谣,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
韩语,1,1,1,0,0,1,1,1,1,1,...,1,1,0,0,1,1,1,1,1,0
儿童,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [8]:
# 开始将tudf 中的数据根据上面的dict_tags中的数据进行替换
tudf = mongotu.changeDataCountTagTUDF(list_name, dict_tags, tudf)
   

In [9]:
tudf.df     

,RuRuUIH,EwNjz,d,RgxKB,ZcO,G,zhZd1J1,79vexmSD,H6fLkSvrlg,c4D,...,PdHcku8A,axRJKQiq5,kRM4zo,DSWI5CjX,FHOCydFdKN,HrlIEqGS,IDDRfNJpw,m5eKRJ,2,gW
小语种,17,5,7,3,1,26,27,7,14,19,...,15,16,5,2,8,22,25,7,19,8
说唱,10,4,4,1,1,10,16,2,8,17,...,3,7,5,0,8,5,12,1,8,3
民族,31,5,17,5,3,26,30,14,32,25,...,19,33,9,4,19,26,28,4,29,4
治愈,14,4,11,4,3,23,20,7,16,17,...,11,18,3,5,8,16,18,3,18,2
感动,23,8,8,7,1,31,32,3,15,28,...,20,13,10,4,12,12,23,4,20,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
兴奋,8,4,3,2,1,7,4,7,7,10,...,6,6,3,1,5,6,10,0,7,1
民谣,26,9,18,5,1,37,28,6,15,34,...,34,22,8,4,19,10,45,7,26,6
韩语,4,3,1,0,0,9,8,5,3,8,...,3,4,0,0,1,4,1,1,5,0
儿童,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [10]:
# 推荐过程
similar_user = tudf.makeSimilarityWithPearson()

In [11]:
similar_user

,RuRuUIH,EwNjz,d,RgxKB,ZcO,G,zhZd1J1,79vexmSD,H6fLkSvrlg,c4D,...,PdHcku8A,axRJKQiq5,kRM4zo,DSWI5CjX,FHOCydFdKN,HrlIEqGS,IDDRfNJpw,m5eKRJ,2,gW
RuRuUIH,1.00,0.96,0.97,0.94,0.84,0.96,0.98,0.96,0.96,0.96,...,0.96,0.96,0.91,0.81,0.97,0.97,0.97,0.95,0.96,0.95
EwNjz,0.96,1.00,0.97,0.94,0.85,0.96,0.97,0.95,0.96,0.95,...,0.97,0.96,0.92,0.83,0.97,0.95,0.97,0.95,0.96,0.93
d,0.97,0.97,1.00,0.93,0.85,0.97,0.98,0.96,0.97,0.96,...,0.97,0.97,0.94,0.82,0.97,0.96,0.98,0.96,0.98,0.95
RgxKB,0.94,0.94,0.93,1.00,0.83,0.93,0.95,0.92,0.94,0.94,...,0.95,0.92,0.92,0.85,0.96,0.93,0.95,0.91,0.94,0.93
ZcO,0.84,0.85,0.85,0.83,1.00,0.86,0.87,0.88,0.89,0.87,...,0.82,0.88,0.82,0.81,0.87,0.86,0.86,0.81,0.85,0.79
G,0.96,0.96,0.97,0.93,0.86,1.00,0.98,0.96,0.97,0.98,...,0.97,0.96,0.94,0.87,0.96,0.97,0.98,0.96,0.98,0.94
zhZd1J1,0.98,0.97,0.98,0.95,0.87,0.98,1.00,0.97,0.98,0.98,...,0.97,0.97,0.95,0.83,0.98,0.97,0.99,0.95,0.98,0.96
79vexmSD,0.96,0.95,0.96,0.92,0.88,0.96,0.97,1.00,0.97,0.96,...,0.95,0.97,0.92,0.84,0.96,0.97,0.97,0.93,0.96,0.93
H6fLkSvrlg,0.96,0.96,0.97,0.94,0.89,0.97,0.98,0.97,1.00,0.98,...,0.96,0.97,0.95,0.85,0.98,0.97,0.98,0.93,0.97,0.94
c4D,0.96,0.95,0.96,0.94,0.87,0.98,0.98,0.96,0.98,1.00,...,0.97,0.95,0.95,0.84,0.98,0.96,0.98,0.92,0.97,0.95


In [12]:
# 获取相似度最高几位用户sign 设置为 1 表示 使用Pearson相关系数获取相关性最高的几个用户
diction = tudf.makeTopNUsers(similar=similar_user,sign=1)
print(diction)
print(len(diction['RuRuUIH']))

{'RuRuUIH': ['Z', 'zhZd1J1', 'uTAbZm', 'GOUkTpak', 'S0JlP3z', '9E', 'KvC', '5u1pJiI', '2ILOvc', 'GcTaF8d', 's7UIjlHxS', 'CO0y', 'fRcKBK16j', 'yNHtXCz5M', 'd', 'UnfVc1Tg', 'zFpXHZ0n', 'FHOCydFdKN', 'HrlIEqGS', 'IDDRfNJpw', 'PdHcku8A', 'axRJKQiq5', '2', 'ere', 'EwNjz', 'MGrVl', '1C3ijya', 'Qu0', 'G', '79vexmSD', 'H6fLkSvrlg', 'w', 'c4D']}
33


In [13]:
# 利用当前用户的信息确定他们的id值形成对应的list_id
diction[mongotu.user_name] = mongotu.makeListIdWithUserName(diction[mongotu.user_name])
print(diction)
print(len(diction[mongotu.user_name]))

{'RuRuUIH': ['1a78ba8eb67911ed95dd00155dadb10b', '1a78c984b67911ed95dd00155dadb10b', '1a78e1eeb67911ed95dd00155dadb10b', '1a78ee28b67911ed95dd00155dadb10b', '1a790340b67911ed95dd00155dadb10b', '1a78fbdeb67911ed95dd00155dadb10b', '1a7904e4b67911ed95dd00155dadb10b', '1a78de4cb67911ed95dd00155dadb10b', '1a78bbceb67911ed95dd00155dadb10b', '1a791bf0b67911ed95dd00155dadb10b', '1a78b80eb67911ed95dd00155dadb10b', '1a7909a8b67911ed95dd00155dadb10b', '1a78c22cb67911ed95dd00155dadb10b', '1a791542b67911ed95dd00155dadb10b', '1a78f4a4b67911ed95dd00155dadb10b', '1a78bc78b67911ed95dd00155dadb10b', '1a78b962b67911ed95dd00155dadb10b', '1a78e6c6b67911ed95dd00155dadb10b', '1a78e2b6b67911ed95dd00155dadb10b', '1a790b06b67911ed95dd00155dadb10b', '1a78ca38b67911ed95dd00155dadb10b', '1a78e7b6b67911ed95dd00155dadb10b', '1a78b6d8b67911ed95dd00155dadb10b', '1a78d0fab67911ed95dd00155dadb10b', '1a78b160b67911ed95dd00155dadb10b', '1a791894b67911ed95dd00155dadb10b', '1a7908eab67911ed95dd00155dadb10b', '1a78dfa0b67911

In [14]:
# 保存过程
# mongotu.saveDataSetRecomAnswer(diction)
# 改变形式，方便存储
docs = mongotu.changeDataFormat(diction=diction, list_keys=list(diction))
print(docs)
# 保存
mongotu.wdb.writeDocument(docs=docs, collection_name="recom")

[{'id': '1a78ac10b67911ed95dd00155dadb10b', 'name': 'RuRuUIH', 'recomUsers': ['1a78ba8eb67911ed95dd00155dadb10b', '1a78c984b67911ed95dd00155dadb10b', '1a78e1eeb67911ed95dd00155dadb10b', '1a78ee28b67911ed95dd00155dadb10b', '1a790340b67911ed95dd00155dadb10b', '1a78fbdeb67911ed95dd00155dadb10b', '1a7904e4b67911ed95dd00155dadb10b', '1a78de4cb67911ed95dd00155dadb10b', '1a78bbceb67911ed95dd00155dadb10b', '1a791bf0b67911ed95dd00155dadb10b', '1a78b80eb67911ed95dd00155dadb10b', '1a7909a8b67911ed95dd00155dadb10b', '1a78c22cb67911ed95dd00155dadb10b', '1a791542b67911ed95dd00155dadb10b', '1a78f4a4b67911ed95dd00155dadb10b', '1a78bc78b67911ed95dd00155dadb10b', '1a78b962b67911ed95dd00155dadb10b', '1a78e6c6b67911ed95dd00155dadb10b', '1a78e2b6b67911ed95dd00155dadb10b', '1a790b06b67911ed95dd00155dadb10b', '1a78ca38b67911ed95dd00155dadb10b', '1a78e7b6b67911ed95dd00155dadb10b', '1a78b6d8b67911ed95dd00155dadb10b', '1a78d0fab67911ed95dd00155dadb10b', '1a78b160b67911ed95dd00155dadb10b', '1a791894b67911ed95dd0